In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import pickle
import time, os

chrome_driver_path = "C:\Metis_Bootcamp\Regression_project\chromedriver.exe"

from bs4 import BeautifulSoup
import requests

import re

import random

In [12]:
header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
}

df_headers = ["Name", "Mileage", "Address", "Rating", "Fuel Type", "City MPG", 
               "Highway MPG", "Drivetrain", "Engine", "Exterior Color", "Interior Color", 
              "Transmission", "Entertainment", "Safety", "Price"]


In [13]:
def get_one_page_urls(page_link):
    
    response = requests.get(page_link, headers=header)

    data = response.text
    
    soup = BeautifulSoup(data, "html.parser")
    
    one_page_urls = []
    
    for i in range(len(soup.find_all(class_="shop-srp-listings__listing-container"))):
        url = soup.find_all(class_="shop-srp-listings__listing-container")[i].find('a')['href']
        one_page_urls.append(f"https://www.cars.com/{url}")
    
    return one_page_urls

In [14]:
def get_all_urls(start_page, num_of_pages, radius, zipcode):
    
    all_urls_list = []
    
    for i in range(num_of_pages):
        
        root_url = "https://www.cars.com/for-sale/searchresults.action/?page={}&perPage=100&rd={}&searchSource=GN_BREADCRUMB&sort=relevance&zc={}".format(i+start_page, radius, zipcode)
        
        all_urls_list.append(get_one_page_urls(root_url))
    
    all_urls = [item for sub_list in all_urls_list for item in sub_list]
    
    return all_urls

In [15]:
def get_car_features(url):
    
    response = requests.get(url, headers=header)

    data = response.text
    
    soup = BeautifulSoup(data, "html.parser")
    
    basic_features = soup.find_all('li', class_='vdp-details-basics__item')
    basic_feature_list = []
    for i in range(len(basic_features)):
        basic_feature_list.append(soup.find_all('li', class_='vdp-details-basics__item')[i].text.strip().split(':'))

    extra_features = soup.find_all(class_='details-feature-list details-feature-list--normalized-features')
    extra_feature_list = []
    for i in range(len(extra_features)):
        extra_feature_list.append(soup.find_all(class_='details-feature-list details-feature-list--normalized-features')[i].text.strip().split('\n\n'))
        
    name = soup.find('h1', class_='cui-heading-2--secondary vehicle-info__title').text
    mileage = soup.find(class_='vdp-cap-price__mileage--mobile vehicle-info__mileage mileage_margin').text.split(" ")[0]
    address = soup.find(class_='get-directions-link seller-details-location__text').text.strip()
    rating = soup.find(class_='rating__link rating__link--has-reviews').text.split('(')[1].split(')')[0]
    fuel_type = -1
    city_MPG = -1
    highway_MPG = -1
    drivetrain = -1
    engine = -1
    exterior_color = -1
    interior_color = -1
    transmission = -1
    entertainment = -1
    safety = -1
    price = soup.find(class_='vehicle-info__price-display vehicle-info__price-display--dealer cui-heading-2').text

    for basic in basic_feature_list:
        if basic[0] == "Fuel Type":
            fuel_type = basic[1]
        if basic[0] == 'City MPG':
            city_MPG = basic[1].split(" ")[1]
        if basic[0] == 'Highway MPG':
            highway_MPG = basic[1].split(" ")[1]
        if basic[0] == 'Drivetrain':
            drivetrain = basic[1]
        if basic[0] == 'Engine':
            engine = basic[1]
        if basic[0] == 'Exterior Color':
            exterior_color = basic[1]
        if basic[0] == 'Interior Color':
            interior_color = basic[1]
        if basic[0] == 'Transmission':
            transmission = basic[1]

    for extra in extra_feature_list:
        if extra[0] == "Entertainment":
            entertainment = extra[1].split('\n')
        if extra[0] == "Safety":
            safety = extra[1].split('\n')
            
    car_dict = dict(zip(df_headers, [name,
                             mileage,
                             address,
                             rating,
                             fuel_type,
                             city_MPG,
                             highway_MPG,
                             drivetrain,
                             engine,
                             exterior_color,
                             interior_color,
                             transmission,
                             entertainment,
                             safety,
                             price]))
 
    return cars_list.append(car_dict)

In [ ]:
start_page = 1

zipcode = 98409 #Tacoma Area

for i in range(5): 
    
    list_of_url = get_all_urls(start_page=start_page, num_of_pages=10, radius=50, zipcode=98409)
    
    cars_list = []

    for url in list_of_url:
        try:
            get_car_features(url)
        except:
            print('..')

    df = pd.DataFrame(cars_list)
    
    df.to_csv('data/df_1000_{}_p{}_p{}.csv'.format(zipcode, start_page, start_page+9))
    
    print("Done for page {} to page {}".format(start_page, start_page+9))
    
    start_page += 10
    
    time.sleep(60)

..
..
..
..
Done for page 1 to page 10
..
..
..
..
..
Done for page 11 to page 20
..
..
..
..
..
..
..
..
..
..


In [32]:
list_of_url = get_all_urls(start_page=11, num_of_pages=20) 

list_of_url_1 = open('data/list_of_url_1.obj', 'wb')
pickle.dump(list_of_url, list_of_url_1)
list_of_url_1.close()

example = open('data/list_of_url_1.obj', 'rb')
exampleObj = pickle.load(example)
example.close()
print(exampleObj)